#### 개요
학습대상 레코드와 컬럼으로 구성된 데이터를 대상으로 전처리 수행

##### 진행사항
아래 내용은 하나의 클래스에서 수행됨
* STEP01: 공사비 데이터셋 전처리(NaN처리, 컬럼추가 등) 
* STEP02: 전주 데이터셋 전처리 및 공사비 데이터셋 병합

##### 향후 추가할 사항
여기서는 전주/전선에 대한 순서를 정하지 않음, 몇가지 컬럼 추가를 수행하지 않음
* aidd-back240313/batch/preprocessing.py 참조

In [1]:
import re
import pandas as pd

import aidd.sys.config as cfg
from aidd.sys.utils import Logs, AiddException
# 이 부분은 IPYNB에서 작업할 경우 만 필요함
# * IPYNB에서 merge.ipynb와 preprocessing.ipynb가 끈어져 추가한 부분임
from aidd.batch.data_manager import get_merged_data, load_pickle, save_pickle

#### 공통 클래스 생성

In [2]:
class Preprocessing():
    def __init__(self, mdata, is_batch=True):
        self._logs = Logs('PROPROCESSING')
        if mdata is None:
            raise AiddException('')
        self.mdata = mdata
        self.is_batch = is_batch
        self.pdf = None
        try:
            self._run()
        except AiddException as ae:
            raise AiddException('PREPROCESSING_ERR', super_err=ae)
        except Exception as e:
            raise AiddException('PREPROCESSING_SERR', super_err=e)
        finally:
            self._logs.stop()
        
    def _run(self):
        self._cons()
        
    def _cons(self):
        logs = Logs('PP_CONS')
        df = self.mdata['CONS']
        logs.mid('SOURCE_SIZE', df.shape)
        
        # 결측치 처리
        df.fillna(0, inplace=True)
        
        # 일자정보 처리
        # * '최종변경일시'를 이용해 다양한 일자정보 컬럼 추가
        df['YEAR'] = df.LAST_MOD_DATE.dt.year
        df['MONTH'] = df.LAST_MOD_DATE.dt.month
        df['DAY'] = df.LAST_MOD_DATE.dt.day
        df['DAYOFWEEK'] = df.LAST_MOD_DATE.dt.dayofweek
        df['DAYOFYEAR'] = df.LAST_MOD_DATE.dt.dayofyear
        df['YEAR_MONTH'] = df.LAST_MOD_DATE.dt.strftime("%Y%m").astype(int)
        
        # 사번정보 처리
        # '최종변경자사번(LAST_MOD_EID)'만 사용(최초등록자==최종변경자사번)
        df['EID_NUMBER'] = df.LAST_MOD_EID.apply(
            lambda x: re.findall(r'\d+', x)[0]
        ).astype(int)
        
        # 사업소명 숫자로 변환
        if self.is_batch:
            olist = df.OFFICE_NAME.unique().tolist()
            save_pickle(olist, pmode='PP_MEM', fcode='OFFICE_LIST')
        else:
            olist = load_pickle(pmode='PP_MEM', fcode='OFFICE_LIST')
        print(olist)
        

In [3]:
bdata = get_merged_data(pmode='MB')
odata = get_merged_data(pmode='MO')

[080d6fdf07da][2024-03-14 17:51:23.787485] 모델 학습용 데이터 불러오기 시작
[080d6fdf07da][2024-03-14 17:51:23.835770]   읽은 데이터: CONS, 데이터 크기: (14729, 11), 처리시간: 0:00:00.048196
[080d6fdf07da][2024-03-14 17:51:23.883647]   읽은 데이터: POLE, 데이터 크기: (27896, 6), 처리시간: 0:00:00.047819
[080d6fdf07da][2024-03-14 17:51:23.993737]   읽은 데이터: LINE, 데이터 크기: (30348, 11), 처리시간: 0:00:00.110013
[080d6fdf07da][2024-03-14 17:51:24.030830]   읽은 데이터: SL, 데이터 크기: (17034, 6), 처리시간: 0:00:00.037011
[080d6fdf07da][2024-03-14 17:51:24.030941] 모델 학습용 데이터 불러오기 종료, 최종 처리시간: 0:00:00.243460
[b1a571a34e73][2024-03-14 17:51:24.031129] 모델 학습용 데이터 불러오기 시작
[b1a571a34e73][2024-03-14 17:51:24.033590]   읽은 데이터: CONS, 데이터 크기: (3, 11), 처리시간: 0:00:00.002428
[b1a571a34e73][2024-03-14 17:51:24.035342]   읽은 데이터: POLE, 데이터 크기: (9, 6), 처리시간: 0:00:00.001712
[b1a571a34e73][2024-03-14 17:51:24.036990]   읽은 데이터: LINE, 데이터 크기: (9, 11), 처리시간: 0:00:00.001611
[b1a571a34e73][2024-03-14 17:51:24.038251]   읽은 데이터: SL, 데이터 크기: (3, 6), 처리시간: 0:00:00.001230
[b1a5

In [4]:
pb = Preprocessing(bdata)
po = Preprocessing(odata, is_batch=False)

[b6aaf2ea83a9][2024-03-14 17:51:24.050883] 데이터 전처리 시작
[f7cfc0e10e40][2024-03-14 17:51:24.051471]   공사비 데이터 전처리 시작
[f7cfc0e10e40][2024-03-14 17:51:24.051511]     공사비+설비갯수 원본 데이터 크기: (14729, 11)
[b6aaf2ea83a9][2024-03-14 17:51:24.057432] 데이터 전처리 종료, 최종 처리시간: 0:00:00.006546


AiddException:   배치 데이터 전처리 과정에서 시스템 에러가 발생했습니다.
  Can only use .dt accessor with datetimelike values
